#NN models

In [1]:
input_path = '../input/tabular-playground-series-apr-2022/'
output_path = './'

In [2]:
import pandas as pd
import numpy as np
import gc
from sklearn.metrics import roc_auc_score

def load_raw_data(train_or_test='train'):
    file_name = f'{input_path}/{train_or_test}.csv'
    df = pd.read_csv(file_name)
    return df

def load_label(train_or_test='train'):
    file_name = input_path + ('train_labels.csv' if train_or_test=='train' else 'sample_submission.csv')
    df = pd.read_csv(file_name)
    return df['state'].values

def competition_metric(y_true, y_score):
    return roc_auc_score(y_true, y_score)

def evaluate(model, X, y):
    return competition_metric(y, model.predict_proba(X)[:, 1])

def to_csv(arr,train_or_test='train',name=None):
    df = pd.DataFrame(arr)
    if type(name)==str:
        df.to_csv(f'{output_path}/{name}_{train_or_test}.csv', index = False )
    else:
        df.to_csv(f'{output_path}/{train_or_test}.csv', index = False )

def ans_to_csv(arr,train_or_test='train',name=None):
    file_name = input_path + ('train_labels.csv' if train_or_test=='train' else 'sample_submission.csv')
    df = pd.read_csv(file_name)
    df['state'] = arr
    if type(name)==str:
        df.to_csv(f'{output_path}/{name}_{train_or_test}.csv', index = False )
    else:
        df.to_csv(f'{output_path}/{train_or_test}.csv', index = False )

def submit(arr):
    df = pd.read_csv(f'{input_path}/sample_submission.csv')
    df['state'] = arr
    df.to_csv(f'{output_path}/submission.csv', index=False)

In [3]:


def group_splitter(df, nfold=5, random_state=None):
    subject_nums = df['subject'].unique()
    rng = np.random.default_rng(random_state)
    subject_to_setnum = rng.integers(0, nfold, subject_nums.shape[0])
    for i in range(nfold):
        val_subjects = subject_nums[subject_to_setnum == i]
        mask_df_val = df['subject'].isin(val_subjects)
        mask_y_val = mask_df_val.iloc[::60]
        yield mask_df_val, mask_y_val

In [4]:
import tensorflow as tf
from tensorflow import keras
from sklearn.base import BaseEstimator, TransformerMixin

class ResNetModel(keras.Model):
    def __init__(self):
        super(ResNetModel, self).__init__()
        self.fns = [
            keras.layers.Conv1D(filters=20, kernel_size=8, padding='same', activation='elu', kernel_regularizer=keras.regularizers.L2(1e-3)), 
            keras.layers.Conv1D(filters=20, kernel_size=8, padding='same', activation='elu', kernel_regularizer=keras.regularizers.L2(1e-3)),
            keras.layers.Conv1D(filters=20, kernel_size=8, padding='same', activation='elu', kernel_regularizer=keras.regularizers.L2(1e-3)),
            keras.layers.AveragePooling1D(2),
            
            keras.layers.Conv1D(filters=20, kernel_size=6, padding='same', activation='elu', kernel_regularizer=keras.regularizers.L2(1e-3)),
            keras.layers.Conv1D(filters=20, kernel_size=6, padding='same', activation='elu', kernel_regularizer=keras.regularizers.L2(1e-3)),
            keras.layers.Conv1D(filters=20, kernel_size=6, padding='same', activation='elu', kernel_regularizer=keras.regularizers.L2(1e-3)),
            keras.layers.AveragePooling1D(2),
            
            keras.layers.Conv1D(filters=20, kernel_size=4, padding='same', activation='elu', kernel_regularizer=keras.regularizers.L2(1e-3)),
            keras.layers.Conv1D(filters=20, kernel_size=4, padding='same', activation='elu', kernel_regularizer=keras.regularizers.L2(1e-3)),
            keras.layers.Conv1D(filters=20, kernel_size=4, padding='same', activation='elu', kernel_regularizer=keras.regularizers.L2(1e-3)),
            keras.layers.AveragePooling1D(3),
            
            keras.layers.GlobalAveragePooling1D(),
            keras.layers.Dense(1, activation='sigmoid')
        ]
        self.bns = [
            keras.layers.BatchNormalization(), 
            keras.layers.BatchNormalization(), 
            
            keras.layers.BatchNormalization(), 
            keras.layers.BatchNormalization(), 
            
            keras.layers.BatchNormalization(), 
            keras.layers.BatchNormalization(), 
            
            keras.layers.BatchNormalization(), 
            keras.layers.BatchNormalization(), 
        ]
        
    def call(self, inputs):
        outputs = inputs
        
        outputs = self.fns[0](outputs)
        res = outputs
        res = self.fns[1](res)
        res = self.fns[2](res)
        outputs += res
        outputs = self.fns[3](outputs)
        
        outputs = self.fns[4](outputs)
        res = outputs
        res = self.fns[5](res)
        res = self.fns[6](res)
        outputs += res
        outputs = self.fns[7](outputs)
        
        outputs = self.fns[8](outputs)
        res = outputs
        res = self.fns[9](res)
        res = self.fns[10](res)
        outputs += res
        outputs = self.fns[11](outputs)
        
        outputs = self.fns[-2](outputs)
        outputs = self.fns[-1](outputs)
        
        return outputs
    
    def predict_proba(self, X):
        return np.concatenate([1-self.predict(X), self.predict(X)], axis=1)

def random_sensor_swap(x, y, random_state=None):
    rng = np.random.default_rng(random_state)
    p_swap = 0.5
    indices = rng.choice(np.arange(x.shape[0]), int(p_swap*x.shape[0]), replace=False)
    x_aug, y_aug = x[indices], y[indices]
    swap_codes = rng.integers(0, 13, (x_aug.shape[0], 2))
    for i in range(x_aug.shape[0]):
        a, b = swap_codes[i]
        x_aug[i, :, [a, b]] = x_aug[i, :, [b, a]]
    x = np.concatenate([x, x_aug], axis=0)
    y = np.concatenate([y, y_aug], axis=0)
    return x, y

def group_splitter(df, nfold=5, random_state=None):
    subject_nums = df['subject'].unique()
    rng = np.random.default_rng(random_state)
    subject_to_setnum = rng.integers(0, nfold, subject_nums.shape[0])
    for i in range(nfold):
        val_subjects = subject_nums[subject_to_setnum == i]
        mask_df_val = df['subject'].isin(val_subjects)
        mask_y_val = mask_df_val.iloc[::60]
        yield mask_df_val, mask_y_val

        
class MyDataAugmenter():
    def fit(self, X, y):
        return self
    
    def transform(self, X, y):
        return X, y

class DF2arr(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return X.loc[:, 'sensor_00':'sensor_12'].values.reshape(-1, 60, 13)
    
    
class MyPreprocessor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X = normalize(X)
        return X
    
def normalize(x):
    x = x / (np.linalg.norm(x, axis=1, keepdims=True) + 1e-10)
    return x

In [5]:
import tensorflow as tf
from tensorflow import keras
from sklearn.base import BaseEstimator, TransformerMixin

class RNNThickModel(keras.Model):
    def __init__(self):
        super(RNNThickModel, self).__init__()
        self.fns = [
            keras.layers.LSTM(
                units=256, 
                kernel_regularizer=keras.regularizers.L2(2e-3),
#                 recurrent_regularizer=keras.regularizers.L2(1e-5),
#                 dropout=0.05,
#                 recurrent_dropout=0.01,
                return_sequences=True
            ),
            keras.layers.LSTM(
                units=128,
                kernel_regularizer=keras.regularizers.L2(2e-3),
#                 recurrent_regularizer=keras.regularizers.L2(1e-5),
#                 dropout=0.05,
#                 recurrent_dropout=0.01,
            ),
            keras.layers.Dense(units=32, activation='elu'),
            keras.layers.Dense(units=1, activation='sigmoid')
        ]
    
    def call(self, inputs):
        outputs = inputs
        for layer in self.fns:
            outputs = layer(outputs)
        return outputs
    
    def predict_proba(self, X):
        return np.concatenate([1-self.predict(X), self.predict(X)], axis=1)

    
class MySoftVoter():
    def __init__(self, models, weights=None):
        self.models = models
        if weights is None:
            weights = np.ones((len(models), ))
        weights /= np.sum(weights)
        self.weights = weights
    
    def predict(self, X):
        result = np.zeros((X.shape[0], ), dtype=X.dtype)
        for model, weight in zip(self.models, self.weights):
            add = model.predict(X)
            if len(add.shape) > 1:
                add = add[:, 0]
            result += add * weight
        return result
    
    def predict_proba(self, X):
        return np.stack([1-self.predict(X), self.predict(X)], axis=1)

In [6]:
def short_test(x,y,n):
    return df.loc[df.sequence<n] ,y[:n]

In [7]:
df = load_raw_data('train')
y = load_label('train')
df_test = load_raw_data("test")
y_test = load_label("test")

In [8]:
class Reverter():
    def __init__(self, random_state=None):
        self.random_state = random_state
        
    def fit(self, X, y):
        return self
    
    def transform(self, X, y):
        rng = np.random.default_rng(self.random_state)
        X_aug, y_aug = X.copy(), y.copy()
        X_aug.iloc[:, 3:16] = -X_aug.iloc[:, 3:16]
        X_aug.iloc[:, 5] = -X_aug.iloc[:, 5]
        X_aug['sequence'] += X['sequence'].max()+1
        X = pd.concat([X, X_aug], axis=0, ignore_index=True)
        y = np.concatenate([y, y_aug], axis=0)
        return X, y


In [9]:
myrev = Reverter()
df,y = myrev.transform(df,y)

In [10]:
from sklearn.pipeline import make_pipeline
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import classification_report

X = DF2arr().transform(df)
subj_nums = df['subject']

preprocessor = make_pipeline(DF2arr(), MyPreprocessor())

keras.backend.clear_session()
tf.random.set_seed(42)

callbacks = [
    keras.callbacks.EarlyStopping(patience=200, restore_best_weights=True)
]
models = [RNNThickModel() for _ in range(6)] + [ResNetModel() for _ in range(6)]
train_preds_array = np.zeros(len(models)*len(y)).reshape(-1,len(models))
test_preds_array = np.zeros(len(models)*len(y_test)).reshape(-1,len(models))
soft_train_preds_vector = np.zeros(len(y))
soft_test_preds_vector = np.zeros(len(y_test))
X_train_F = preprocessor.transform(df)
X_test_F = preprocessor.transform(df_test)
for mask_df_val, mask_y_val in group_splitter(df, nfold=5, random_state=42):
    models = [RNNThickModel() for _ in range(6)] + [ResNetModel() for _ in range(6)]
    df_train, y_train = df[~mask_df_val], y[~mask_y_val]
    df_val, y_val = df[mask_df_val], y[mask_y_val]
    X_train = preprocessor.fit_transform(df_train)
    X_val = preprocessor.transform(df_val)
    print(X_train.shape, X_val.shape)
    weights = []
    for model,i in zip(models,range(len(models))):
        with tf.device('gpu:0'):
                model.compile(
                    loss='binary_crossentropy', 
                    metrics=['AUC'],
                    optimizer=keras.optimizers.Adam(1e-3))
                model.fit(
                    X_train, y_train, 
                    batch_size=1024,
                    epochs=500, 
                    callbacks=callbacks,
                    validation_data=(X_val, y_val),
                    verbose=0
                )
                weights.append( model.evaluate(X_val, y_val)[-1] - 0.5 )
        train_preds_array[:,i] += model.predict_proba(X_train_F)[:,1]/5
        test_preds_array[:,i] += model.predict_proba(X_test_F)[:,1]/5
        print(model)
        print(evaluate(model, X_train, y_train))
        print(evaluate(model, X_val, y_val))
        print(classification_report(y_val, (model.predict(X_val) >= 0.5).astype(int), digits=4 ))
    model_soft = MySoftVoter(models, weights)
    print(evaluate(model_soft,X_train,y_train))
    print(evaluate(model_soft, X_val,y_val))
    soft_train_preds_vector += model_soft.predict_proba(X_train_F)[:,1]/5
    soft_test_preds_vector += model_soft.predict_proba(X_test_F)[:,1]/5
    del models
    gc.collect()

to_csv(train_preds_array,"train")
to_csv(test_preds_array,"test")
to_csv(soft_train_preds_vector,"train",name="soft")
to_csv(soft_test_preds_vector,"test",name="soft")

2022-10-19 09:24:28.538784: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-19 09:24:28.616653: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-19 09:24:28.617435: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-19 09:24:28.619294: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

(41634, 60, 13) (10302, 60, 13)


2022-10-19 09:24:32.556459: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-10-19 09:24:36.089410: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


322/322 [==============================] - 2s 5ms/step - loss: 0.2755 - auc: 0.9620
0.9759081461518657
0.962046814016374
              precision    recall  f1-score   support

           0     0.8907    0.9005    0.8955      5184
           1     0.8980    0.8880    0.8930      5118

    accuracy                         0.8943     10302
   macro avg     0.8944    0.8943    0.8943     10302
weighted avg     0.8943    0.8943    0.8943     10302

322/322 [==============================] - 2s 5ms/step - loss: 0.2573 - auc: 0.9678
0.9821978593010495
0.9680676467466556
              precision    recall  f1-score   support

           0     0.8997    0.9099    0.9048      5184
           1     0.9077    0.8972    0.9024      5118

    accuracy                         0.9036     10302
   macro avg     0.9037    0.9036    0.9036     10302
weighted avg     0.9037    0.9036    0.9036     10302

322/322 [==============================] - 2s 6ms/step - loss: 0.2510 - auc: 0.9704
0.9825588158679706


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0     0.5245    1.0000    0.6881      4824
           1     0.0000    0.0000    0.0000      4374

    accuracy                         0.5245      9198
   macro avg     0.2622    0.5000    0.3440      9198
weighted avg     0.2751    0.5245    0.3609      9198

288/288 [==============================] - 1s 5ms/step - loss: 0.2469 - auc: 0.9703
0.9818968396689929
0.970393564489699
              precision    recall  f1-score   support

           0     0.9162    0.9156    0.9159      4824
           1     0.9070    0.9076    0.9073      4374

    accuracy                         0.9118      9198
   macro avg     0.9116    0.9116    0.9116      9198
weighted avg     0.9118    0.9118    0.9118      9198

288/288 [==============================] - 1s 5ms/step - loss: 0.2784 - auc: 0.9624
0.9818831687460663
0.9625068530234061
              precision    recall  f1-score   support

           0     0.8999    0.9075    0.9037     